<a href="https://colab.research.google.com/github/AndresJejen/Neumonia-Vision-3D/blob/master/ProyectoVision_de_maquina%2C_Detecci%C3%B3n_de_Neumonia_usando_Vision_de_Maquina_en_imagenes_de_Rayos_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identificando casos de Neumonía con herramientas de visión de maquina.

* Hair Sebastian López Criollo - hslopezc@unal.edu
* German Andrés Jején Cortés - gjejen@unal.edu.co

En este Notebook de constante actualización, encontrarás el proceso de recolección y análisis de imagenes de rayos x de pulmones diagnosticados como sanos o con neunomia. Nuestro objetivo es desarrollar una solución que ayude a profesionales de la salud a identificar con mayor rapidez casos de esta enfermedad para dar un tratamiento oportuno.

El conjunto de datos Lo estamos descargando de Kaggle, usando la API que provee la plataforma.

In [0]:
# !mkdir -p ~/.kaggle
# !cp "./drive/My Drive/Colab Notebooks/Proyect Machine Vision/kaggle.json" ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip /content/chest-xray-pneumonia.zip -d "./drive/My Drive/Colab Notebooks/Proyect Machine Vision/"

In [0]:
import os
import glob
import h5py
import shutil
import imgaug as aug # Image Augmentation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
import cv2
color = sns.color_palette()
%matplotlib inline

In [0]:
# Definir la ruta de las imagenes
data_dir = Path('./drive/My Drive/Colab Notebooks/Proyect Machine Vision/chest_xray/chest_xray')

# Ruta del directorio de Imagenes de entrenamiento
train_dir = data_dir / 'train'

# Ruta del directorio de Imagenes de Validación
val_dir = data_dir / 'val'

# Ruta del directorio de Imagenes de Prueba
test_dir = data_dir / 'test'

Los datos no se encuentran etiquetados, es por esto que basandonos en el nombre provisto por el archivo generamos una lista de etiquetas, 1 para casos de neumonía y 0 para casos normales.

In [0]:
# Ruta del directorio de Imagenes de entrenamiento tanto Normales como con neumonia
normal_cases_dir = train_dir / 'NORMAL'
pneumonia_cases_dir = train_dir / 'PNEUMONIA'

# Lista de todas las imagenes 
normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')

# Arreglo vacio apra insertar los datos en una lista en el siguiente formato (img_path, label)
train_data = []

# Ir a través de todos los casos normales. La etiqueta para estos casos será 0
for img in normal_cases:
    train_data.append((img,0))

# Ir a través de todos los casos de neumonía. La etiqueta para estos casos será 1
for img in pneumonia_cases:
    train_data.append((img, 1))

# Obtenga un marco de datos de pandas de los datos que tenemos en nuestra lista
train_data = pd.DataFrame(train_data, columns=['image', 'label'],index=None)

# revolver los datos
train_data = train_data.sample(frac=1.).reset_index(drop=True)

# un vistazo a los datos
train_data.head()

## Analizando los datos
Trabajar con imagenes que puedan ser procesadas por una visipon computacional trae problemas comunes, como es el caso de no tener la sufuciente cantidad de datos o que se encuentren mas ejemplares de unas clases que de otras. Para confirmar si estos escenarios están presenten en nuestro proyecto verificamos las etiquetas, realizando un conteo por clase.

In [0]:
# Contar por cada clase cuantas imagenes Existen
cases_count = train_data['label'].value_counts()
print(cases_count)

# Graficar los resultados
plt.figure(figsize=(10,8))
sns.barplot(x=cases_count.index, y= cases_count.values)
plt.title('Number of cases', fontsize=14)
plt.xlabel('Case type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(range(len(cases_count.index)), ['Normal(0)', 'Pneumonia(1)'])
plt.show()

Existe una gran diferencia entre casos diagnosticados y casos normales. Es necesario conseguir mas datos o usar herramientas de Data Augmentation.

In [0]:
# obtener unas pocas muestras de ambas clases 
pneumonia_samples = (train_data[train_data['label']==1]['image'].iloc[:5]).tolist()
normal_samples = (train_data[train_data['label']==0]['image'].iloc[:5]).tolist()

# Concatenar los datos en una simple lista y borrar las listas de arriba
samples = pneumonia_samples + normal_samples
del pneumonia_samples, normal_samples

# Mostrar los ejemplos
f, ax = plt.subplots(2,5, figsize=(30,10))
for i in range(10):
    img = imread(samples[i])
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i<5:
        ax[i//5, i%5].set_title("Pneumonia")
    else:
        ax[i//5, i%5].set_title("Normal")
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()

Podemos observar unos pocos ejemplos de imagenes, es curioso observar como para personas sin experiencia médica como nosostros es dificil encontrar diferencias significativas entre ambos casos.


In [0]:
# Realizamos el mismo proceso de captura de datos del directorio de validación
Get the path to the sub-directories
normal_cases_dir = val_dir / 'NORMAL'
pneumonia_cases_dir = val_dir / 'PNEUMONIA'

normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')

valid_data = []
valid_labels = []

# Some images are in grayscale while majority of them contains 3 channels. So, if the image is grayscale, we will convert into a image with 3 channels.
# We will normalize the pixel values and resizing all the images to 224x224 

# Normal cases
for img in normal_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(0, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)
                      
# Pneumonia cases        
for img in pneumonia_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(1, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)
    
# Convert the list into numpy arrays
valid_data = np.array(valid_data)
valid_labels = np.array(valid_labels)

print("Total number of validation examples: ", valid_data.shape)
print("Total number of labels:", valid_labels.shape)

In [0]:
# Sequencia de Aumentación de datos
seq = iaa.OneOf([
    iaa.Fliplr(), # Rotación Horizontal
    iaa.Affine(rotate=20), # Rotación de la imagen
    iaa.Multiply((1.2, 1.5))]) # Cambiar el brillo aleatoriamente

### Optimización de la Memoria
Resulta que trabajamos con un conjunto de datos que tiene un peso de aproximadamente una giga, además queremos hacer buen uso de recursos como la memoria RAM de la laquina en donde trabajamos. Para esto usamos una estrategia de Generador de datos que toma de manera aleatoria la ruta de la imagen y las carga a un dataframe temporar que sera utilizado en cada proceso que se realize, asi no es necesario tener cargado en memoria RAM todas las imagenes esperando a ser procesadas.

In [0]:
def data_gen(data, batch_size):
    n = len(data)
    steps = n//batch_size
    
    batch_data = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)
    batch_labels = np.zeros((batch_size,2), dtype=np.float32)

    indices = np.arange(n)
  
    i =0
    while True:
        np.random.shuffle(indices)
        count = 0
        next_batch = indices[(i*batch_size):(i+1)*batch_size]
        for j, idx in enumerate(next_batch):
            img_name = data.iloc[idx]['image']
            label = data.iloc[idx]['label']
            
            encoded_label = to_categorical(label, num_classes=2)
            img = cv2.imread(str(img_name))
            img = cv2.resize(img, (224,224))
          
            if img.shape[2]==1:
                img = np.dstack([img, img, img])
            
            orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            orig_img = img.astype(np.float32)/255.
            
            batch_data[count] = orig_img
            batch_labels[count] = encoded_label
            
            if label==0 and count < batch_size-2:
                aug_img1 = seq.augment_image(img)
                aug_img2 = seq.augment_image(img)
                aug_img1 = cv2.cvtColor(aug_img1, cv2.COLOR_BGR2RGB)
                aug_img2 = cv2.cvtColor(aug_img2, cv2.COLOR_BGR2RGB)
                aug_img1 = aug_img1.astype(np.float32)/255.
                aug_img2 = aug_img2.astype(np.float32)/255.

                batch_data[count+1] = aug_img1
                batch_labels[count+1] = encoded_label
                batch_data[count+2] = aug_img2
                batch_labels[count+2] = encoded_label
                count +=2
            
            else:
                count+=1
            
            if count==batch_size-1:
                break
            
        i+=1
        yield batch_data, batch_labels
            
        if i>=steps:
            i=0

In [0]:
# Generador de los datos, ademas de aumentar las muestras por paquete de imagenes.
train_data_gen = data_gen(data=train_data, batch_size=10)

In [0]:
train_data_gen